[![image-1.png](https://i.postimg.cc/Vk3VrVXM/image-1.png)](https://postimg.cc/v4tzCXqH)

<div style="padding:15px 20px 20px 20px;
            color:#004346;
            font-size:40px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400">
<p style="font-weight: bold; text-align: center;">Прогнозирование оттока клиентов</p>

</div>

<div style="padding:0px 40px 30px;
            color:#004346;
            font-size:110%;
            display:fill;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:450;">
    
__Заказчик:__ Оператор связи «ТелеДом».
    
__Описание задачи:__ Для борьбы с оттоком клиентов, сотрудники компании будут предлагать промокоды и специальные условия тем абонентам, которые имеют намерение прекратить пользоваться услугами связи. Это позволит заранее выявлять пользователей, склонных к расторжению договора.

__Постановка задачи:__ Разработать модель для прогнозирования вероятности расторжения договора абонентом.
    
__Оценка качества модели:__ Метрики ROC-AUC, значение должно быть не менее 0.85. 

__Описание услуг компании:__ Оператор предоставляет два основных типа услуг:
    
- Стационарную телефонную связь. Телефон можно подключить к нескольким линиям одновременно.
- Интернет. Подключение бывает двух типов: через телефонную линию DSL (англ. digital subscriber line — «цифровая абонентская линия») или оптоволоконный кабель (англ. fiber optic).
    
Также абонентам доступен ряд дополнительных услуг:
    
- Интернет-безопасность: антивирус (Device Protection) и блокировка опасных сайтов (Online Security);
- Выделенная линия технической поддержки (Tech Support);
- Облачное хранилище файлов для резервного копирования данных (Online Backup);
- Стриминговое телевидение (Streaming TV) и каталог фильмов (Streaming Movies).
    
За услуги клиенты могут платить ежемесячно или раз в 1–2 года. Доступны различные способы расчёта и возможность получить электронный чек.
    
__Описание данных:__ персональные данные о некоторых клиентах, информация об их тарифах и услугах.
 
Данные хранятся в базе данных PostgreSQL. Она состоит из нескольких [таблиц](https://github.com/Denis-Mukhanov/client-churn-prediction/blob/main/data/data%20description.pdf):
    
- `contract` — информация о договорах;
- `personal` — персональные данные клиентов;
- `internet` — информация об интернет-услугах;
- `phone` — информация об услугах телефонии.


</div>    

<div style="padding:0px 20px 10px;
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400">

# Используемые библиотеки

</div>

In [1]:
from rich.console import Console
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

import phik
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine

# константы
RANDOM_SEED = 181023

# настройки блокнота
pd.options.display.max_colwidth = 500
random.seed(RANDOM_SEED)

# экземпляр консоли
console = Console()

## Подключение к базе данных

In [2]:
# конфигурация для подключения к базе данных
db_config = {
    'user': 'praktikum_student',# имя пользователя,
    'pwd': 'Sdf4$2;d-d30pp',# пароль,
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,# порт подключения,
    'db': 'data-science-final'# название базы данных,
} 

# создание подключения к базе данных
connection = psycopg2.connect(
    user=db_config['user'],
    password=db_config['pwd'],
    host=db_config['host'],
    port=db_config['port'],
    database=db_config['db']
)

# создаем курсора для выполнения запросов
cursor = connection.cursor()

__функция для выполнения запросов__

In [3]:
# функция для выполнения запросов
def request(query):
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        result_df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
        return result_df
    except Exception as e:
        print(f'Error: {e}')
        connection.rollback()  # откат транзакции, для избежания блокировки

In [4]:
# тестовый запрос
query = '''
SELECT *
FROM telecom.contract
LIMIT 1;
'''
display(request(query))

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,4472-LVYGI,2020-02-01,None,Two year,Yes,Bank transfer (automatic),52.55,None


<div style="padding:0px 20px 10px;
            color:#004346;
            font-size:15px;
            display:fill;
            text-align:center;
            border-radius:20px;
            border: 5px double;
            border-color:#201E20;
            background-color: #E8F1F2;
            overflow:hidden;
            font-weight:400">

# Первичное исследование таблиц базы данных

</div>

__Проверим таблицы на наличие данных__

In [14]:
tables = ["contract", "personal", "internet", "phone"]

In [16]:
for table in tables:
    query = f"SELECT COUNT(*) FROM telecom.{table};"
    contract_count = request(query).iloc[0][0]
    print(f"Количество записей в таблице {table}:", contract_count)


Количество записей в таблице contract: 7043
Количество записей в таблице personal: 7043
Количество записей в таблице internet: 5518
Количество записей в таблице phone: 6362


>Все четыре таблицы доступны и содержат данные, 
>Количество записей  в таблицах "contract" и "personal" одинаковы, что соответствует условию задачи,
>Таблицы "internet" и "phone" имеют различное количество записей, что может быть связани, что не все пользователь пользуются одновременно интернет и услугами телефонии,
>Количество записей невилико и мы можем выгрузить таблицы, для дальнейшего анализа.

## Выгрузка таблиц базы данных

In [72]:
# информация о договорах
contract_df = request("SELECT * FROM telecom.contract;")
print('Датасет с информацией о договорах')
display(contract_df.head(2))
display(contract_df.info())
# персональные данные клиентов
personal_df = request("SELECT * FROM telecom.personal;")
print('Датасет с информацией о персональных данные клиентов')
display(personal_df.head(2))
display(personal_df.info())
# информация об интернет-услугах
internet_df = request("SELECT * FROM telecom.internet;")
internet_df.columns = list(internet_df.iloc[0].values)
internet_df = internet_df.drop(0, axis=0)
print('Датасет с информацией об интернет-услугах')
display(internet_df.head(2))
display(internet_df.info())
# информация об услугах телефонии
phone_df = request("SELECT * FROM telecom.phone;")
phone_df.columns = list(phone_df.iloc[0].values)
phone_df = phone_df.drop(0, axis=0)
print('Датасет с информацией об услугах телефонии')
display(phone_df.head(2))
display(phone_df.info())

Датасет с информацией о договорах


,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,4472-LVYGI,2020-02-01,None,Two year,Yes,Bank transfer (automatic),52.55,None
1,3115-CZMZD,2020-02-01,None,Two year,No,Mailed check,20.25,None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customerID        7043 non-null   object
 1   BeginDate         7043 non-null   object
 2   EndDate           1869 non-null   object
 3   Type              7043 non-null   object
 4   PaperlessBilling  7043 non-null   object
 5   PaymentMethod     7043 non-null   object
 6   MonthlyCharges    7043 non-null   object
 7   TotalCharges      7032 non-null   object
dtypes: object(8)
memory usage: 440.3+ KB


None

Датасет с информацией о персональных данные клиентов


...

None

Датасет с информацией об услугах телефонии


,customerID,MultipleLines
1,5575-GNVDE,No
2,3668-QPYBK,No


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6361 entries, 1 to 6361
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customerID     6361 non-null   object
 1   MultipleLines  6361 non-null   object
dtypes: object(2)
memory usage: 99.5+ KB


None

In [73]:
# список датасетов
list_df = [contract_df, personal_df, internet_df, phone_df]

>Регистр названий признаков не соответствует стандарту.

__Перееименуем признаки__

In [74]:
new_column_names = {
    'customerID': 'customer_id',
    'BeginDate': 'begin_date',
    'EndDate': 'end_date',
    'Type': 'type',
    'PaperlessBilling': 'paperless_billing',
    'PaymentMethod': 'payment_method',
    'MonthlyCharges': 'monthly_charges',
    'TotalCharges': 'total_charges',
    'SeniorCitizen': 'senior_citizen',
    'Partner': 'partner',
    'Dependents': 'dependents',
    'InternetService': 'internet_service',
    'OnlineSecurity': 'online_security',
    'OnlineBackup': 'online_backup',
    'DeviceProtection': 'device_protection',
    'TechSupport': 'tech_support',
    'StreamingTV': 'streaming_tv', 
    'StreamingMovies': 'streaming_movies',
    'MultipleLines': 'multiplelines'
}

In [79]:
for df in list_df:
    df.globals = df.rename(columns=new_column_names)

/var/folders/1l/cj5j7tnj6q347f0pj242vpfm0000gn/T/ipykernel_55126/1130311398.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.globals = df.rename(columns=new_column_names)
/var/folders/1l/cj5j7tnj6q347f0pj242vpfm0000gn/T/ipykernel_55126/1130311398.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.globals = df.rename(columns=new_column_names)
/var/folders/1l/cj5j7tnj6q347f0pj242vpfm0000gn/T/ipykernel_55126/1130311398.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.globals = df.rename(columns=new_column_names)
/var/folders/1l/cj5j7tnj6q347f0pj242vpfm0000gn/T/ipykernel_55126/1130311398.py:2: UserWarning: Pandas doesn't al

In [77]:
df

,customer_id,multiplelines
1,5575-GNVDE,No
2,3668-QPYBK,No
3,9237-HQITU,No
4,9305-CDSKC,Yes
5,1452-KIOVK,Yes
...,...,...
6357,2569-WGERO,No
6358,6840-RESVB,Yes
6359,2234-XADUH,Yes
6360,8361-LTMKD,Yes
